In [ ]:
#imports
import numpy as np
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
#synthetic dataset(1200 records)
np.random.seed(42)

N = 1200  # dataset size

sleep = np.random.uniform(4, 9, N)              # hours/night
screen = np.random.uniform(1, 10, N)             # hours/day
study = np.random.uniform(0, 8, N)               # hours/day
attendance = np.random.uniform(50, 100, N)       # %
stress = np.random.uniform(1, 10, N)             # scale
typing_speed = np.random.uniform(2, 6, N)        # chars/sec
typing_error = np.random.uniform(0, 15, N)       # %
pause = np.random.uniform(150, 500, N)           # ms

# Non-linear target (this justifies MLP)
focus_score = (
    12*sleep
    - 4*screen
    + 6*study
    + 0.3*attendance
    - 5*stress
    + 4*typing_speed
    - 2*typing_error
    - 0.01*pause
)

# Add noise
focus_score += np.random.normal(0, 8, N)
focus_score = np.clip(focus_score, 0, 100)

X = np.column_stack([
    sleep, screen, study, attendance,
    stress, typing_speed, typing_error, pause
])

y = focus_score / 100.0  # normalize target

In [ ]:
#Normalize inputs
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#MLP Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

history = model.fit(
    X_scaled, y,
    epochs=60,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/60


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1198 - mae: 0.2897 - val_loss: 0.0718 - val_mae: 0.2294
Epoch 2/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0614 - mae: 0.2099 - val_loss: 0.0429 - val_mae: 0.1726
Epoch 3/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0376 - mae: 0.1610 - val_loss: 0.0261 - val_mae: 0.1337
Epoch 4/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0238 - mae: 0.1279 - val_loss: 0.0160 - val_mae: 0.1053
Epoch 5/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0148 - mae: 0.0995 - val_loss: 0.0112 - val_mae: 0.0871
Epoch 6/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0120 - mae: 0.0885 - val_loss: 0.0092 - val_mae: 0.0772
Epoch 7/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0095 - mae: 0.0785 - val_loss: 0.0082 - val_mae: 0.0720
Epoch 8/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0094 - mae: 0.0764 - val_loss: 0.0077 - val_mae: 0.0695
Epoch 9/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0080 - mae: 0.0704 - va

In [ ]:
#UI with Proper units (measuring / metrics)
features = [
    ("Sleep Hours (hrs/night)", 4, 9, 7),
    ("Screen Time (hrs/day)", 1, 10, 5),
    ("Study Hours (hrs/day)", 0, 8, 4),
    ("Attendance (%)", 50, 100, 80),
    ("Stress Level (1–10)", 1, 10, 5),
    ("Typing Speed (chars/sec)", 2, 6, 4),
    ("Typing Error (%)", 0, 15, 5),
    ("Pause Time (ms)", 150, 500, 300)
]

sliders = []
for name, mn, mx, val in features:
    s = widgets.FloatSlider(
        description=name,
        min=mn, max=mx, value=val,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    sliders.append(s)
    display(s)

output_box = widgets.Output()

FloatSlider(value=7.0, description='Sleep Hours (hrs/night)', layout=Layout(width='100%'), max=9.0, min=4.0, s…

FloatSlider(value=5.0, description='Screen Time (hrs/day)', layout=Layout(width='100%'), max=10.0, min=1.0, st…

FloatSlider(value=4.0, description='Study Hours (hrs/day)', layout=Layout(width='100%'), max=8.0, style=Slider…

FloatSlider(value=80.0, description='Attendance (%)', layout=Layout(width='100%'), min=50.0, style=SliderStyle…

FloatSlider(value=5.0, description='Stress Level (1–10)', layout=Layout(width='100%'), max=10.0, min=1.0, styl…

FloatSlider(value=4.0, description='Typing Speed (chars/sec)', layout=Layout(width='100%'), max=6.0, min=2.0, …

FloatSlider(value=5.0, description='Typing Error (%)', layout=Layout(width='100%'), max=15.0, style=SliderStyl…

FloatSlider(value=300.0, description='Pause Time (ms)', layout=Layout(width='100%'), max=500.0, min=150.0, sty…

In [ ]:
#Prediction function
def predict_focus(btn):
    with output_box:
        clear_output()
        values = np.array([[s.value for s in sliders]])
        values_scaled = scaler.transform(values)

        pred = model.predict(values_scaled, verbose=0)[0][0] * 100

        if pred > 70:
            risk = "🟢 LOW Burnout Risk"
        elif pred > 45:
            risk = "🟡 MODERATE Burnout Risk"
        else:
            risk = "🔴 HIGH Burnout Risk"

        print(f"🎯 Focus Score: {pred:.1f}%")
        print(f"🔥 Burnout Risk: {risk}")
        print("\nExplanation:")
        print("• Model combines lifestyle + typing behavior")
        print("• Nonlinear interactions learned via hidden layers")

In [ ]:
#Button & display
btn = widgets.Button(description="🔮 Estimate Focus", button_style='success')
btn.on_click(predict_focus)

display(btn)
display(output_box)

Button(button_style='success', description='🔮 Estimate Focus', style=ButtonStyle())

Output()